In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from rich.progress import track
from scipy.spatial.distance import cosine
from itertools import product
from rich.progress import track

rng = np.random.default_rng(0)

In [4]:
with h5py.File("/media/scratch/sageev-midi/20250320/specdiff.h5", "r") as hdf:
    print(hdf.keys())
    print(hdf["embeddings"])
    print(str(hdf["filenames"][0], "utf-8"))
    print(hdf["embeddings"][:5])

<KeysViewHDF5 ['embeddings', 'filenames']>


KeyError: 'Unable to synchronously open object (invalid dataset size, likely file corruption)'

In [6]:
# p_h5 = "../data/tables/20250110/specdiff.h5"
# with h5py.File(p_h5, "r") as hdf_in:
#     with h5py.File("../data/tables/20250110/specdiff_new.h5", "w") as hdf_out:
#         new_names = hdf_out.create_dataset(
#             "filenames",
#             (len(hdf_in["filenames"]), 1),
#             dtype=h5py.string_dtype(encoding="utf-8"),
#             fillvalue="",
#         )
#         embeddings = hdf_out.create_dataset(
#             "embeddings", (len(hdf_in["embeddings"]), 768)
#         )

#         for i, name in track(enumerate(hdf_in["filenames"])):
#             new_names[i] = str(name[0][:-7], "utf-8")
#             embeddings[i] = hdf_in["embeddings"][i]

Output()

In [2]:
with h5py.File("../data/tables/20250110/specdiff.h5", "r") as hdf:
    print(hdf["filenames"][:5])
    print(hdf["embeddings"][:5])
with h5py.File("../data/tables/20250110/pitch_histogram.h5", "r") as hdf:
    print(hdf["filenames"][:5])
    print(hdf["histograms"][:5])

[[b'20231220-080-01_0000-0005_t00s00']
 [b'20231220-080-01_0000-0005_t00s01']
 [b'20231220-080-01_0000-0005_t00s02']
 [b'20231220-080-01_0000-0005_t00s03']
 [b'20231220-080-01_0000-0005_t00s04']]
[[-0.0511964  -0.14626487  0.06894483 ...  0.08640549 -0.02314547
  -0.11110201]
 [-0.04755304 -0.10617626  0.10255402 ...  0.0741595  -0.02618582
  -0.10119651]
 [-0.05469938 -0.12968346  0.0791863  ...  0.0878297  -0.02015256
  -0.10488336]
 [-0.04319933 -0.11640533  0.08603172 ...  0.08256292 -0.01906523
  -0.09960214]
 [-0.05720589 -0.14236337  0.06919699 ...  0.09306504 -0.02204188
  -0.10629459]]


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../data/tables/20250110/pitch_histogram.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
# verify hdf5 file
p_h5 = "../data/tables/20250320/neighbors.h5"
with h5py.File(p_h5, "r") as hdf:
    print(f"{len(hdf.keys())} datasets in file")
    df = pd.read_hdf(p_h5, key="neighbors")
df.tail()

1 datasets in file


,prev_2,prev,current,next,next_2
20250219-066-03_0508-0516,20250219-066-03_0494-0501,20250219-066-03_0501-0508,20250219-066-03_0508-0516,20250219-066-03_0516-0523,20250219-066-03_0523-0530
20250219-066-03_0516-0523,20250219-066-03_0501-0508,20250219-066-03_0508-0516,20250219-066-03_0516-0523,20250219-066-03_0523-0530,20250219-066-03_0530-0538
20250219-066-03_0523-0530,20250219-066-03_0508-0516,20250219-066-03_0516-0523,20250219-066-03_0523-0530,20250219-066-03_0530-0538,20250219-066-03_0538-0545
20250219-066-03_0530-0538,20250219-066-03_0516-0523,20250219-066-03_0523-0530,20250219-066-03_0530-0538,20250219-066-03_0538-0545,None
20250219-066-03_0538-0545,20250219-066-03_0523-0530,20250219-066-03_0530-0538,20250219-066-03_0538-0545,None,None


In [ ]:
# load prompt dataset
p_test = "../data/datasets/test/dataset samples"
transformations = [f"t{t:02d}s{s:02d}" for t, s in product(range(12), range(8))]
q_files = [f"{f[:-4]}_{rng.choice(transformations)}" for f in os.listdir(p_test)]
q_files[:3]

In [ ]:
# load prompt embeddings
q_embeddings = [embedding_dataset[key] for key in q_files]
q_embeddings[0]

In [ ]:
embedding_dataset.values()

In [ ]:
# find best and worst n matches for prompts
n_matches = 3
indices = []  # To store the indices of the best and worst matches
for q_emb in q_embeddings:
    similarities = []
    for idx, dataset in enumerate(embedding_dataset.values()):
        sim = 1 - cosine(q_emb, dataset)
        similarities.append((sim, idx))

    similarities.sort(key=lambda x: x[0], reverse=True)  # Highest first
    best_indices = [idx for _, idx in similarities[:n_matches]]  # Top 3
    worst_indices = [idx for _, idx in similarities[-n_matches:]]  # Bottom 3

    indices.append((best_indices, worst_indices))

In [ ]:
embedding_dataset.close()